# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/nose_poke_identifier'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [8]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data"
OUTPUT_DIR = os.path.join(git_root, "proc") # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [9]:
COLS_TO_KEEP = ['session_dir', 'recording', 'metadata_dir', 'metadata_file',
'original_file', 'filename', 'session_path', 'all_subjects',
       'current_subject', 'event_timestamps', 'video_name',
       'video_timestamps', 'event_frames', 'first_item_data']

In [10]:
# TODO: Find way not to hard code this
ALL_SESSION_DIR = glob.glob(os.path.join(git_root, "data/*.rec"))


In [11]:
ALL_SESSION_DIR

['/nancy/projects/nose_poke_identifier/data/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

In [12]:
def recursive_dictionary():
    return defaultdict(recursive_dictionary)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [13]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [14]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = recursive_dictionary()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1
Skipping file 20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.timestampoffset.txt due to error: Settings format not supported


/nancy/projects/nose_poke_identifier/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.timestampoffset.txt due to error: Settings format not supported


In [15]:
session_to_trodes_data

defaultdict(<function __main__.recursive_dictionary()>,
            {'20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1': defaultdict(dict,
                         {'20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged': {'timestampoffset': {},
                           'DIO': {'dio_ECU_Dout1': {'description': 'State change data for one digital channel. Display_order is 1-based',
                             'byte_order': 'little endian',
                             'original_file': '20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firm

- Adding the video timestamps

In [16]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            if "video_timestamps" not in session_to_trodes_data[session_basename][session_basename]:
                session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1
Current Video Name: 20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.videoTimeStamps.cameraHWSync
Current Video Name: 20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2.videoTimeStamps.cameraHWSync


In [17]:
session_to_trodes_data[session_basename][session_basename]["video_timestamps"]

defaultdict(dict,
            {'1': {'clock rate': '30000',
              'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
              'data': array([( 7978450, 0, 0), ( 7979510, 0, 0), ( 7979835, 0, 0), ...,
                     (76317064, 0, 0), (76318450, 0, 0), (76318450, 0, 0)],
                    dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
              'filename': '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.videoTimeStamps.cameraHWSync'},
             '2': {'clock rate': '30000',
              'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
              'data': array([( 7978450, 0, 0), ( 7979597, 0, 0), ( 7979835, 0, 0), ...,
                     (76317064, 0, 0), (76317064, 0, 0), (76318450, 0, 0)],
                    dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')]),
              'filename': '20230612_112630_standard_comp_to_t

- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [18]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [19]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,direction,id,display_order,fields,data,filename,decimation,clock rate,session_path
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,output,ECU_Dout1,2,<time uint32><state uint8>,"[[7977066, 0]]",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Dout2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,output,ECU_Dout2,3,<time uint32><state uint8>,"[[7977066, 0]]",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,input,ECU_Din4,9,<time uint32><state uint8>,"[[7977066, 0]]",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,input,ECU_Din3,8,<time uint32><state uint8>,"[[7977066, 0], [41014777, 1], [41015772, 0], [...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...
4,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,input,ECU_Din2,6,<time uint32><state uint8>,"[[7977066, 0], [8169811, 1], [8226416, 0], [83...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...


In [20]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_timestamp,direction,id,display_order,fields,data,filename,decimation,clock rate,session_path
21,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,NaN,NaN,NaN,<time uint32>,"[[7977066], [7977067], [7977068], [7977069], [...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...
22,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,coordinates,Pad locations in microns,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,NaN,NaN,NaN,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...
23,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,time,timestamps,Timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,7977066,NaN,NaN,NaN,<time uint32><systime int64>,"[[7977066, 1686583861038711900], [7977067, 168...",20230612_112630_standard_comp_to_training_D1_s...,20,NaN,/nancy/projects/nose_poke_identifier/data/2023...
24,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979510, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...
25,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979597, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...


- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [21]:
trodes_metadata_df["data"].iloc[0]

array([(7977066, 0)], dtype=[('time', '<u4'), ('state', 'u1')])

In [22]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [23]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [24]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,fields,data,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[7977066, 0]]",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,[7977066],state,[0]
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Dout2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[7977066, 0]]",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,[7977066],state,[0]
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[7977066, 0]]",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,[7977066],state,[0]
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[7977066, 0], [41014777, 1], [41015772, 0], [...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 41014777, 41015772, 41241974, 412473...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
4,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><state uint8>,"[[7977066, 0], [8169811, 1], [8226416, 0], [83...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."


In [25]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,fields,data,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
21,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32>,"[[7977066], [7977067], [7977068], [7977069], [...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ..."
22,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,coordinates,Pad locations in microns,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
23,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,time,timestamps,Timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,<time uint32><systime int64>,"[[7977066, 1686583861038711900], [7977067, 168...",20230612_112630_standard_comp_to_training_D1_s...,20,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",systime,"[1686583861038711900, 1686583861038713500, 168..."
24,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979510, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...,PosTimestamp,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
25,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[7978450, 0, 0], [7979597, 0, 0], [7979835, 0...",20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...,PosTimestamp,"[7978450, 7979597, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [26]:
trodes_metadata_df["recording"].unique()

array(['20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1'],
      dtype=object)

## Getting the subject information from the metadata

In [27]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["recording"].apply(lambda x: x.replace("-","_").split("subj")[-1].split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted([i.strip().strip(".") for i in x]))

In [28]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["all_subjects"].apply(lambda x: x[0] if len(x) == 1 else np.nan)

In [29]:
trodes_metadata_df["all_subjects"]

0          [1.1]
1          [1.1]
2          [1.1]
3          [1.1]
4          [1.1]
5          [1.1]
6          [1.1]
7          [1.1]
8          [1.1]
9          [1.1]
10         [1.1]
11         [1.1]
12         [1.2]
13         [1.2]
14         [1.2]
15         [1.2]
16         [1.2]
17         [1.2]
18         [1.2]
19         [1.2]
20         [1.2]
21         [1.2]
22         [1.2]
23         [1.2]
24    [1.1, 1.2]
25    [1.1, 1.2]
Name: all_subjects, dtype: object

In [30]:
trodes_metadata_df["current_subject"]

0     1.1
1     1.1
2     1.1
3     1.1
4     1.1
5     1.1
6     1.1
7     1.1
8     1.1
9     1.1
10    1.1
11    1.1
12    1.2
13    1.2
14    1.2
15    1.2
16    1.2
17    1.2
18    1.2
19    1.2
20    1.2
21    1.2
22    1.2
23    1.2
24    NaN
25    NaN
Name: current_subject, dtype: object

In [31]:
ALL_SUBJECTS_COL = "all_subjects"

## Dropping all the rows with unneeded metadata

In [32]:
trodes_metadata_df["metadata_dir"].unique()

array(['DIO', 'raw', 'analog', 'time', 'video_timestamps'], dtype=object)

In [33]:
METADATA_TO_KEEP = ['raw', 'DIO', 'video_timestamps']

In [34]:
trodes_metadata_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(METADATA_TO_KEEP)]

In [35]:
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("out")]
trodes_metadata_df = trodes_metadata_df[~trodes_metadata_df["metadata_file"].str.contains("coordinates")]


In [36]:
trodes_metadata_df = trodes_metadata_df.reset_index(drop=True)

# Getting the first time stamp of each recording

In [37]:
trodes_metadata_df["metadata_dir"].unique()

array(['DIO', 'raw', 'video_timestamps'], dtype=object)

In [38]:
trodes_raw_df = trodes_metadata_df[(trodes_metadata_df["metadata_dir"] == "raw") & (trodes_metadata_df["metadata_file"] == "timestamps")].copy()


In [39]:
trodes_raw_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
4,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",[1.1],1.1
9,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",[1.2],1.2


In [40]:
trodes_raw_df["first_timestamp"] = trodes_raw_df["first_item_data"].apply(lambda x: x[0])

In [41]:
recording_to_first_timestamp = trodes_raw_df.set_index('session_dir')['first_timestamp'].to_dict()

In [42]:
recording_to_first_timestamp

{'20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1': 7977066}

#TODO: Make sure that the first timestamp is the same for all the recordings in the session

In [43]:
trodes_metadata_df["first_timestamp"] = trodes_metadata_df["session_dir"].map(recording_to_first_timestamp)

In [44]:
trodes_metadata_df["first_timestamp"]

0     7977066
1     7977066
2     7977066
3     7977066
4     7977066
5     7977066
6     7977066
7     7977066
8     7977066
9     7977066
10    7977066
11    7977066
Name: first_timestamp, dtype: int64

# Getting the event timestamps

In [45]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,[7977066],state,[0],[1.1],1.1
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din3,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 41014777, 41015772, 41241974, 412473...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
4,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",[1.1],1.1


In [46]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
7,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,[7977066],state,[0],[1.2],1.2
8,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
9,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,raw,timestamps,Raw timestamps,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",time,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",[1.2],1.2
10,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...,PosTimestamp,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN
11,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...,PosTimestamp,"[7978450, 7979597, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN


In [47]:
# trodes_state_df = trodes_metadata_df[trodes_metadata_df["last_dtype_name"] == "state"].copy()

# Filtering for digital IO channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"].isin(["DIO"])].copy()
# Filtering for tone and port entry related channels
trodes_state_df = trodes_metadata_df[trodes_metadata_df["id"].isin(["ECU_Din1", "ECU_Din2"])].copy()


In [48]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1
5,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2
8,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2


In [49]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: np.column_stack([np.where(x["last_item_data"] == 1)[0], np.where(x["last_item_data"] == 1)[0]+1]), axis=1)

In [50]:
trodes_state_df["event_indexes"] = trodes_state_df.apply(lambda x: x["event_indexes"][x["event_indexes"][:, 1] <= x["first_item_data"].shape[0] - 1], axis=1)

In [51]:
trodes_state_df["event_timestamps"] = trodes_state_df.apply(lambda x: x["first_item_data"][x["event_indexes"]], axis=1)

In [52]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8169811, 8226416], [8366813, 8384208], [8894..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[9103808, 9303807], [11503806, 11703806], [13..."
5,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[9103808, 9303807], [11503806, 11703806], [13..."
8,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,NaN,/nancy/projects/nose_poke_identifier/data/2023...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8169811, 8226416], [8366813, 8384208], [8894..."


## Updating the video timestamps

## Syncing up the video frame data

In [53]:
# Getting the rows that are the metadata for the video timestamps
trodes_video_df = trodes_metadata_df[trodes_metadata_df["metadata_dir"] == "video_timestamps"].copy().reset_index(drop=True)

# Filtering for the first video only
# This only applies to this pilot data where we are only looking the at competition data
trodes_video_df = trodes_video_df[trodes_video_df["metadata_file"] == "1"].copy()

In [54]:
trodes_video_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,filename,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,video_timestamps,1,NaN,NaN,NaN,NaN,NaN,NaN,...,20230612_112630_standard_comp_to_training_D1_s...,NaN,30000,/nancy/projects/nose_poke_identifier/data/2023...,PosTimestamp,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1.1, 1.2]",NaN


In [55]:
# Making the video timestamps into an evenly distributed array
trodes_video_df["video_timestamps"] = trodes_video_df["first_item_data"]

In [56]:
# Removing the columns that are no longer needed
trodes_video_df = trodes_video_df[["filename", "video_timestamps", "session_dir"]].copy()

In [57]:
# Renaming the filename so that we can merge with other dataframes with the same column name
trodes_video_df = trodes_video_df.rename(columns={"filename": "video_name"})

In [58]:
trodes_video_df

,video_name,video_timestamps,session_dir
0,20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...",20230612_112630_standard_comp_to_training_D1_s...


- Adding each video as a row to each state row

In [59]:
trodes_state_df = pd.merge(trodes_state_df, trodes_video_df, on=["session_dir"], how="left")

In [60]:
trodes_state_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_dtype_name,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8169811, 8226416], [8366813, 8384208], [8894...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ..."
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[9103808, 9303807], [11503806, 11703806], [13...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ..."
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,time,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[9103808, 9303807], [11503806, 11703806], [13...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,time,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8169811, 8226416], [8366813, 8384208], [8894...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ..."


## Finding the closest frame to each event

In [61]:
def find_closest_indices(array1, array2):
    array1_flat = array1.flatten()
    indices = np.array([np.abs(array2 - num).argmin() for num in array1_flat])
    return indices.reshape(array1.shape)

In [62]:
trodes_state_df["event_timestamps"].iloc[1]

array([[ 9103808,  9303807],
       [11503806, 11703806],
       [13803806, 14003803],
       [15603802, 15803801],
       [16703800, 16903800],
       [17703799, 17903799],
       [19603797, 19803797],
       [21103796, 21303796],
       [23203793, 23403793],
       [24203798, 24403793],
       [25803789, 26003788],
       [27103792, 27303790],
       [28403788, 28603788],
       [29403787, 29603787],
       [30803786, 31003786],
       [32703784, 32903783],
       [34303782, 34503784],
       [35703781, 35903780],
       [37903778, 38103778],
       [39103777, 39303777],
       [45103770, 45303770],
       [46303769, 46503769],
       [47403768, 47603767],
       [48703767, 48903766],
       [49803763, 50003765],
       [50803764, 51003764],
       [51803763, 52003762],
       [53903761, 54103761],
       [56303761, 56503758],
       [58703756, 58903755],
       [60903756, 61103755],
       [62603751, 62803748],
       [65003749, 65203748],
       [66203747, 66403747],
       [675037

In [63]:
trodes_state_df["event_frames"] = trodes_state_df.apply(lambda x: find_closest_indices(x["event_timestamps"], x["video_timestamps"]), axis=1)

In [64]:
trodes_state_df

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,first_item_data,last_dtype_name,last_item_data,all_subjects,current_subject,event_indexes,event_timestamps,video_name,video_timestamps,event_frames
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8169811, 8226416], [8366813, 8384208], [8894...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[192, 248], [389, 405], [916, 929], [929, 948..."
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.1],1.1,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[9103808, 9303807], [11503806, 11703806], [13...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[1125, 1324], [3519, 3720], [5815, 6014], [76..."
2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,"[7977066, 9103808, 9303807, 11503806, 11703806...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[9103808, 9303807], [11503806, 11703806], [13...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[1125, 1324], [3519, 3720], [5815, 6014], [76..."
3,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,DIO,dio_ECU_Din2,State change data for one digital channel. Dis...,little endian,20230612_112630_standard_comp_to_training_D1_s...,20000,2.4.0,May 24 2023,...,"[7977066, 8169811, 8226416, 8366813, 8384208, ...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",[1.2],1.2,"[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11,...","[[8169811, 8226416], [8366813, 8384208], [8894...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[192, 248], [389, 405], [916, 929], [929, 948..."


## Combine raw and state dataframes

In [65]:
RAW_COLS_TO_KEEP = ['session_dir',
 'recording',
 'original_file',
 'filename',
 'session_path',
 'current_subject',
 'first_item_data']

In [66]:
STATE_COLS_TO_KEEP = ['session_dir',
 'metadata_file',
 'event_timestamps',
 'video_name',
 'video_timestamps',
 'event_frames',]

In [67]:
trodes_state_df = trodes_state_df[STATE_COLS_TO_KEEP].drop_duplicates(subset=["session_dir", "metadata_file"]).sort_values(["session_dir", "metadata_file"]).reset_index(drop=True).copy()

In [68]:
same_columns = ['session_dir', 'video_name']
different_columns = ['metadata_file', 'event_frames', 'event_timestamps']

In [69]:
trodes_state_df = trodes_state_df.groupby(same_columns).agg({**{col: 'first' for col in trodes_state_df.columns if col not in same_columns + different_columns}, **{col: lambda x: x.tolist() for col in different_columns}}).reset_index()

In [70]:
trodes_state_df["tone_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[0])
trodes_state_df["port_entry_timestamps"] = trodes_state_df["event_timestamps"].apply(lambda x: x[1])

trodes_state_df["tone_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[0])
trodes_state_df["port_entry_frames"] = trodes_state_df["event_frames"].apply(lambda x: x[1])


In [71]:
trodes_state_df = trodes_state_df.drop(columns=["event_timestamps", "event_frames"], errors="ignore")

In [72]:
trodes_state_df

,session_dir,video_name,video_timestamps,metadata_file,tone_timestamps,port_entry_timestamps,tone_frames,port_entry_frames
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[dio_ECU_Din1, dio_ECU_Din2]","[[9103808, 9303807], [11503806, 11703806], [13...","[[8169811, 8226416], [8366813, 8384208], [8894...","[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..."


In [73]:
trodes_raw_df = trodes_raw_df[RAW_COLS_TO_KEEP].reset_index(drop=True).copy()

In [74]:
trodes_raw_df

,session_dir,recording,original_file,filename,session_path,current_subject,first_item_data
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/nancy/projects/nose_poke_identifier/data/2023...,1.1,"[7977066, 7977067, 7977068, 7977069, 7977070, ..."
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/nancy/projects/nose_poke_identifier/data/2023...,1.2,"[7977066, 7977067, 7977068, 7977069, 7977070, ..."


In [75]:
trodes_final_df = pd.merge(trodes_raw_df, trodes_state_df, on=["session_dir"], how="left")

In [76]:
trodes_final_df = trodes_final_df.rename(columns={"first_item_data": "raw_timestamps"})
trodes_final_df = trodes_final_df.drop(columns=["metadata_file"], errors="ignore")
trodes_final_df = trodes_final_df.sort_values(["session_dir", "recording"]).reset_index(drop=True).copy()

In [77]:
trodes_final_df

,session_dir,recording,original_file,filename,session_path,current_subject,raw_timestamps,video_name,video_timestamps,tone_timestamps,port_entry_timestamps,tone_frames,port_entry_frames
0,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/nancy/projects/nose_poke_identifier/data/2023...,1.1,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[9103808, 9303807], [11503806, 11703806], [13...","[[8169811, 8226416], [8366813, 8384208], [8894...","[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..."
1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,/nancy/projects/nose_poke_identifier/data/2023...,1.2,"[7977066, 7977067, 7977068, 7977069, 7977070, ...",20230612_112630_standard_comp_to_training_D1_s...,"[7978450, 7979510, 7979835, 7981221, 7982607, ...","[[9103808, 9303807], [11503806, 11703806], [13...","[[8169811, 8226416], [8366813, 8384208], [8894...","[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..."


In [78]:
trodes_final_df.to_pickle(os.path.join(OUTPUT_DIR, "{}_trodes_metadata.pkl".format(OUTPUT_PREFIX)))

In [79]:
OUTPUT_DIR

'/nancy/projects/nose_poke_identifier/proc'